## Demo of ct_schools

This input spreadsheet has schools spelled slightly differently from the way they appear in most state databases.

Let's use the ct_schools package to convert them to a common spelling. We'll find that this involves some "guessing" and isn't perfect. We can determine how good our guesses are and focus on fixing just the weakest guesses. So there's always going to be some manual work, but a lot less.

In this walkthrough, we will:

1. Import the messy excel sheet sheff_16_17.xlsx 
2. Play around with ct_schools.closest 
3. Add a column of name guesses
4. Add a column of each guess's fuzzy ratio 
5. Find the questionable matches

In [62]:
import pandas as pd
from ct_schools.schools import closest

### 1. Import the source data

Here we pull the correct sheet from the excel workbook, and skip a few rows. 

The school column contains the school names, spelled inconsistently compared with other data sets. 

In [63]:
# Here I do a bunch of cleaning in one step to get this data set down to a manageable number of nicely named columns.
# Describing these steps isn't the point of this notebook.
df = pd.read_excel("sheff_16_17.xlsx",
                   sheetname="DesegStandard_SheffSchools_All",
                   skiprows=3,
                   usecols=[1,8,9,10],
                   names=["school","students_total","students_bl","students_ri"])\
.dropna(subset=["school"])

df.head()

,school,students_total,students_bl,students_ri
2,The Global Experience Magnet School,218,151,67
3,Wintonbury Early Childhood Magnet,323,181,142
4,Connecticut IB Academy,197,102,95
5,Betances Early Reading Lab,295,195,100
6,*Betances STEM Magnet,375,284,91


### 2. Play around with ct_schools.closest

ct_schools.closest uses a common list of ct schools that has some other info joined in, like school ID, that makes it really usefully for pulling in other school data.

Below shows the output of a search for "Valley." The closest school found is called "Valley View School"

In [64]:
closest("Valley")

,School Name,Organization Type,Organization Code,Location 1,Zipcode,Phone,PreKindergarten,Kindergarten,Grade 1,Grade 2,...,Grade 9,Grade 10,Grade 11,Grade 12,Student Open Date,Interdistrict Magnet,lat_lng,lat,lng,FUZZ_RATIO
267,Valley View School,Public Schools,1130211,"81 High St.\nPortland, CT 06480\n(41.57868, -7...",06480,860-342-3131,1,1,1,0,...,0,0,0,0,01Jul1984 0:00:00.000,0.0,"('41.57868', ' -72.626149')",41.57868,-72.626149,50


We can get more than one result, say the closest five schools...

In [65]:
closest("Valley",lim=5)

,School Name,Organization Type,Organization Code,Location 1,Zipcode,Phone,PreKindergarten,Kindergarten,Grade 1,Grade 2,...,Grade 9,Grade 10,Grade 11,Grade 12,Student Open Date,Interdistrict Magnet,lat_lng,lat,lng,FUZZ_RATIO
267,Valley View School,Public Schools,1130211,"81 High St.\nPortland, CT 06480\n(41.57868, -7...",06480,860-342-3131,1,1,1,0,...,0,0,0,0,01Jul1984 0:00:00.000,0.0,"('41.57868', ' -72.626149')",41.578680,-72.626149,50
441,Smalley Academy,Public Schools,891211,"175 West St.\nNew Britain, CT 06051-1415\n(41....",06051-1415,860-225-8647,0,1,1,1,...,0,0,0,0,01Jul1984 0:00:00.000,0.0,"('41.677627', ' -72.771328')",41.677627,-72.771328,48
40,River Valley Services,State Agency Facilities,3370815,"P.O. Box 351, Leak Hall\nMiddletown, CT 06457\n",06457,860-262-7009,0,0,0,0,...,1,1,1,1,01Jul2007 0:00:00.000,0.0,NaN,NaN,NaN,44
1178,Shepaug Valley School,Regional Schools,2120312,"159 South Street\nWashington, CT 06793-0000\n(...",06793-0000,860-868-7326,0,0,0,0,...,1,1,1,1,26Aug2013 0:00:00.000,NaN,"('41.601715', ' -73.303104')",41.601715,-73.303104,44
268,Pleasant Valley School,Public Schools,1320211,"591 Ellington Rd.\nSouth Windsor, CT 06074-411...",06074-4118,860-610-0291,0,1,1,1,...,0,0,0,0,01Jul1984 0:00:00.000,0.0,"('41.811222', ' -72.597108')",41.811222,-72.597108,43


### 3. Adding a name_guess column to our original sheet

Now that we know how to use the closest() function, let's apply it to the entire dataset. 

I'll use a lambda function on the input column (df["school"])and call closest. 

Since closest returns an entire DataFrame, and we only want the "School Name" value from the first row, we'll index into it with the bracket notation.

In [66]:
df["guess_name"] = df["school"].apply(lambda x: closest(x).iloc[0]["School Name"])
df.head()

,school,students_total,students_bl,students_ri,guess_name
2,The Global Experience Magnet School,218,151,67,Global Experience Magnet School
3,Wintonbury Early Childhood Magnet,323,181,142,Wintonbury Early Childhood Magnet School
4,Connecticut IB Academy,197,102,95,Connecticut IB Academy
5,Betances Early Reading Lab,295,195,100,Betances Early Reading Lab Magnet School
6,*Betances STEM Magnet,375,284,91,Betances STEM Magnet School


### 4. Add a column with fuzz values

ct_schools.closest() relies on the fuzzywuzzy python library to compute how close our guesses are. Fuzzywuzzy produces a score from 0 to 100, with 100 being a perfect match. 

We want to see those scores so we can identify any questionable guesses in the next section.

I'll use the same approach as in step 3, but isolate the "FUZZ_RATIO" column.

In [67]:
df["fuzz"] = df["school"].apply(lambda x: closest(x).iloc[0]["FUZZ_RATIO"])
df.head()

,school,students_total,students_bl,students_ri,guess_name,fuzz
2,The Global Experience Magnet School,218,151,67,Global Experience Magnet School,94
3,Wintonbury Early Childhood Magnet,323,181,142,Wintonbury Early Childhood Magnet School,90
4,Connecticut IB Academy,197,102,95,Connecticut IB Academy,100
5,Betances Early Reading Lab,295,195,100,Betances Early Reading Lab Magnet School,79
6,*Betances STEM Magnet,375,284,91,Betances STEM Magnet School,83


Now, I'll sort by fuzz scores to identify any that are really low (potentially bad guesses).

In [68]:
df.sort_values(by="fuzz").head(20)

,school,students_total,students_bl,students_ri,guess_name,fuzz
29,GHAA [Full-Day],469,304,165,Path Academy,37
30,GHAA [Half-Day],242,69,173,Nathan Hale-Ray High School,43
32,GHAMAS [Half-Day],43,8,35,Chapman School,45
35,****Medical Professions and Teacher Preparatio...,574,431,143,Charter Oak International Academy,52
71,*Bloomfield Agriscience and Technology Center,NaN,131,+,Bloomfield School District,54
31,GHAA/Middle,327,238,89,RHAM Middle School,55
72,Glastonbury Agriscience and Technology Center,89,28,61,Glastonbury School District,56
73,Suffield Regional Agriscience Center,118,9,109,Suffield School District,57
7,**Breakthrough II,272,219,53,Breakthrough II Magnet School,65
36,MLC Magnet Global/International Studies,688,515,173,Metropolitan Learning Center for Global and In...,67


Note how the first 15 or so results need to be checked over by hand, but then they get pretty reliable.

Let's take a look at the last 20 results, with the strongest fuzz scores.

In [69]:
df.sort_values(by="fuzz").tail(20)

,school,students_total,students_bl,students_ri,guess_name,fuzz
39,*Public Safety Academy,474,359,115,Public Safety Academy,98
9,**Capital Community College Magnet Academy,56,47,9,Capital Community College Magnet Academy,98
22,Sport and Medical Sciences Academy,709,488,221,Sports and Medical Sciences Academy,99
62,*A. I. Prince Technical High School,719,655,64,A. I. Prince Technical High School,99
26,Academy of Aerospace and Engineering Elementary,389,212,177,Academy of Aerospace and Engineering Elementary,100
4,Connecticut IB Academy,197,102,95,Connecticut IB Academy,100
65,Vinal Technical High School,522,158,364,Vinal Technical High School,100
64,Howell Cheney Technical High School,624,234,390,Howell Cheney Technical High School,100
63,E. C. Goodwin Technical High School,624,419,205,E. C. Goodwin Technical High School,100
14,Hartford Magnet Trinity College Academy,1024,768,256,Hartford Magnet Trinity College Academy,100


But wait, you say, most of those are exact matches. That's not fair.

OK, let's drop all the exact matches and look at the next best matches.

In [70]:
df[df["fuzz"] < 100].sort_values(by="fuzz").tail(20)

,school,students_total,students_bl,students_ri,guess_name,fuzz
12,Environmental Sciences @ Mary M. Hooker,611,416,195,Environmental Sciences Magnet at Hooker School,78
5,Betances Early Reading Lab,295,195,100,Betances Early Reading Lab Magnet School,79
10,**Capital Preparatory Magnet Academy,554,462,92,Capital Preparatory Magnet School,81
17,Montessori Magnet at Annie Fisher,336,193,143,Montessori Magnet at Fisher School,81
51,*Dr. James H Naylor/CCSU Leadership Academy,643,495,148,Naylor/CCSU Leadership Academy,82
6,*Betances STEM Magnet,375,284,91,Betances STEM Magnet School,83
13,Great Path Academy,272,184,88,Great Path Academy at MCC,84
33,Glastonbury-East Hartford Magnet,452,250,202,Glastonbury/East Hartford Magnet School,87
37,Montessori Magnet School (CREC),349,236,113,Montessori Magnet School,87
18,Montessori Magnet at Moylan,304,206,98,Montessori Magnet at Moylan School,89


Pretty good, right?